In [ ]:
from anytree import Node, RenderTree
from anytree.exporter import UniqueDotExporter

In [84]:
# one use case is to number the instructions and give each one a unique ID
#      so if one instruction involves a value, then the value actually already maps to the 
#.     instruction in the original sequence
class Abstract_Node:
    #value_id = 0
    def __init__(self, value_id, cur_operation_name, children_lst, is_leaf_value_node):
        if is_leaf_value_node:
            assert children_lst is None
            assert cur_operation_name is None
            
            assert value_id is not None
            assert is_leaf_value_node is not None
            
            self.cur_node = Node(value_id)
        else:
            assert value_id is None
            
            assert cur_operation_name is not None
            assert children_lst is not None
            assert is_leaf_value_node is not None
            
            self.cur_node = Node(cur_operation_name,  children=children_lst)
    
    def __repr__(self):
        return repr(self.cur_node)
    
    def print_node(self):
        for pre, fill, node in RenderTree(self.cur_node):
             print("%s%s" % (pre, node.name))
                
    def to_png(self, png_name):
        UniqueDotExporter(self.cur_node).to_picture(png_name + ".png")

In [3]:
def write_to_file(contents, path):
    with open(path, 'w') as f:
        f.write(contents)
    
def read_from_file(path):
    with open(path) as f:
        return ''.join(f.readlines())
    
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll-1
    return -9999, -9999

In [105]:
class Emulator:
    
    def __init__(self):
        # right most is the top of the stack
        self.stack = []
        # [Abstract_Value() for i in range(5)]
        # self.max_length = 10000
        

    
    def pop(self):
        assert len(self.stack) >= 1
        self.stack.pop()
        
    def dupX(self, X):
        # 1 <= X <= 16
        assert len(self.stack) >= X
        self.stack.append(self.stack[-X])
        
    def swapX(self, X):
        assert len(self.stack) >= X + 1
        top_item = self.stack.pop()
        temp = self.stack[-X]
        self.stack[-X] = top_item
        self.stack.append(temp)  
        
    def mstore(self):
        1
        
    def logX(self, X, event_sequence):
        
        children_lst = []
        for _ in range(X + 2):
            children_lst.append(self.stack.pop().cur_node)
        
        event_sequence.append(children_lst)

        
    def abstract_remove_3_add_1(self, instruction_name):
        assert len(self.stack) >= 3

        top_item1 = self.stack.pop().cur_node
        top_item2 = self.stack.pop().cur_node
        top_item3 = self.stack.pop().cur_node

        new_expression = Abstract_Node(None, instruction_name, [top_item1, top_item2, top_item3], False)
        self.stack.append(new_expression)
        
    def abstract_remove_2_add_1(self, instruction_name):
        assert len(self.stack) >= 2

        top_item1 = self.stack.pop().cur_node
        top_item2 = self.stack.pop().cur_node

        new_expression = Abstract_Node(None, instruction_name, [top_item1, top_item2], False)
        self.stack.append(new_expression)

    def abstract_remove_1_add_1(self, instruction_name):
        assert len(self.stack) >= 1

        top_item1 = self.stack.pop().cur_node

        new_expression = Abstract_Node(None, instruction_name, [top_item1], False)
        self.stack.append(new_expression)

    def abstract_remove_None_add_1(self, instruction_name, value_id):
        
        new_expression = Abstract_Node(value_id, None, None, True)
        self.stack.append(new_expression)


In [24]:
import os
# os.system('solc -o ./compiled_opcodes/output --opcodes ./code_repository/RewardsCollector.sol')

In [26]:
opcodes = read_from_file('./compiled_opcodes/RouterImmutables.txt')

constants_removed_seq = []
for i in opcodes.split(' '):
    if not i.startswith('0x') and len(i) != 0:
        constants_removed_seq.append(i)

In [31]:
len(constants_removed_seq)

972

In [103]:
abstract_remove_3_add_1 = ['CREATE']


remove_2_add_1_lst = ['ADD', 'AND', 'MUL', 'SUB', 'DIV', 'SDIV', 'LT',
                      'GT', 'EQ', 'MOD', 'OR', 'SHR', 'SLT', 'SUB']

remove_1_add_1_lst = ['BALANCE', 'CALLDATALOAD', 'EXTCODESIZE', 'ISZERO', 'MLOAD', 'NOT']

remove_None_add_1_lst = ['CODESIZE', 'MSIZE', 'CALLVALUE']


stack_emulator = Emulator()

constants_removed_seq_with_ind = [(i, constants_removed_seq[i]) for i in range(len(constants_removed_seq))]

# note: to inherit a previous stack for permutation testing, we just rerun the sequence up
#.   to the current instruction

event_sequence = []
simulated_memory = dict()


for t in constants_removed_seq_with_ind:
    i = t[0]
    cur_inst = t[1]
    print(t)
    
    if cur_inst in memory_operations:
        print('new stack initiating')
        print(stack_emulator.stack)
        stack_emulator = Emulator()
        continue
    
    if cur_inst.startswith('PUSH'):
        stack_emulator.abstract_remove_None_add_1(cur_inst, i)
    
    elif cur_inst.startswith('DUP'):
        dup_num = int(cur_inst[3:])
        stack_emulator.dupX(dup_num)
        
    elif cur_inst.startswith('LOG'):
        log_num = int(cur_inst[3:])
        stack_emulator.logX(dup_num, event_sequence)
        
        
    elif cur_inst in remove_2_add_1_lst:
        stack_emulator.abstract_remove_2_add_1(cur_inst)
        
    elif cur_inst in remove_1_add_1_lst:
        stack_emulator.abstract_remove_1_add_1(cur_inst)
        
    elif cur_inst in remove_None_add_1_lst:
        stack_emulator.abstract_remove_None_add_1(cur_inst, i)
    else:
        print('ddfdsffsfs',t)
    
    print(stack_emulator.stack)

(0, 'PUSH2')
[Node('/0')]
(1, 'PUSH1')
[Node('/0'), Node('/1')]
(2, 'MSTORE')
new stack initiating
[Node('/0'), Node('/1')]
(3, 'CALLVALUE')
[Node('/3')]
(4, 'DUP1')
[Node('/3'), Node('/3')]
(5, 'ISZERO')
[Node('/ISZERO/3'), Node('/ISZERO')]
(6, 'PUSH2')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6')]
(7, 'JUMPI')
ddfdsffsfs (7, 'JUMPI')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6')]
(8, 'PUSH1')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6'), Node('/8')]
(9, 'DUP1')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6'), Node('/8'), Node('/8')]
(10, 'REVERT')
ddfdsffsfs (10, 'REVERT')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6'), Node('/8'), Node('/8')]
(11, 'JUMPDEST')
ddfdsffsfs (11, 'JUMPDEST')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6'), Node('/8'), Node('/8')]
(12, 'POP')
ddfdsffsfs (12, 'POP')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6'), Node('/8'), Node('/8')]
(13, 'PUSH1')
[Node('/ISZERO/3'), Node('/ISZERO'), Node('/6'), Node('/8'), Node('/8'), Node('/13')]
(14, 'MLOAD

AssertionError: 

In [39]:
stack_emulator.stack

In [ ]:
# Expressions touching the memory: CALLVALUE, CREATE, LOG2, MSTORE
# Other expressions that should also be set as a seperator: INVALID
# Logical branching statements: JUMP, JUMPDEST, JUMPI
# Not sure about: KECCAK256, RETURN, REVERT, 

# Remove 2, add 1 to the stack: ADD, AND, MUL, SUB, DIV, SDIV, LT, GT, EQ, MOD, OR, SHR, SLT,
        # SUB
# Remove 1, add 1 to the stack: BALANCE, CALLDATALOAD, EXTCODESIZE, ISZERO, MLOAD, NOT, 
# Remove None, add 1 to the stack: CODESIZE, MSIZE, 

# implemented/known: POP, PUSH, SWAP

In [28]:
set(constants_removed_seq)

{'ADD',
 'AND',
 'BALANCE',
 'CALLDATALOAD',
 'CALLVALUE',
 'CODECOPY',
 'CODESIZE',
 'CREATE',
 'DUP1',
 'DUP2',
 'DUP3',
 'DUP4',
 'DUP5',
 'DUP6',
 'DUP8',
 'EQ',
 'EXTCODESIZE',
 'GT',
 'INVALID',
 'ISZERO',
 'JUMP',
 'JUMPDEST',
 'JUMPI',
 'KECCAK256',
 'LOG2',
 'LT',
 'MLOAD',
 'MOD',
 'MSIZE',
 'MSTORE',
 'NOT',
 'OR',
 'POP',
 'PUSH1',
 'PUSH17',
 'PUSH2',
 'PUSH20',
 'PUSH22',
 'PUSH32',
 'PUSH5',
 'PUSH7',
 'PUSH8',
 'RETURN',
 'REVERT',
 'SHR',
 'SLT',
 'SUB',
 'SWAP1',
 'SWAP16',
 'SWAP2',
 'SWAP3'}

In [13]:
opcode_sequence = read_from_file('opcode_repository/RouterImmutables.txt')

constants_removed_seq = []
for i in opcode_sequence.split(' '):
    if not i.startswith('0x') and len(i) != 0:
        constants_removed_seq.append(i)

In [14]:
set(constants_removed_seq)

{'ADD',
 'AND',
 'BALANCE',
 'CALLDATALOAD',
 'CALLVALUE',
 'CODECOPY',
 'CODESIZE',
 'CREATE',
 'DUP1',
 'DUP2',
 'DUP3',
 'DUP4',
 'DUP5',
 'DUP6',
 'DUP8',
 'EQ',
 'EXTCODESIZE',
 'GT',
 'INVALID',
 'ISZERO',
 'JUMP',
 'JUMPDEST',
 'JUMPI',
 'KECCAK256',
 'LOG2',
 'LT',
 'MLOAD',
 'MOD',
 'MSIZE',
 'MSTORE',
 'NOT',
 'OR',
 'POP',
 'PUSH1',
 'PUSH17',
 'PUSH2',
 'PUSH20',
 'PUSH22',
 'PUSH32',
 'PUSH5',
 'PUSH7',
 'PUSH8',
 'RETURN',
 'REVERT',
 'SHR',
 'SLT',
 'SUB',
 'SWAP1',
 'SWAP16',
 'SWAP2',
 'SWAP3'}

In [197]:
def readable_stack(stack):
    new_rep = []
    for i in range(len(stack)):
        if type(stack[i]) == Abstract_Value:
            new_rep.append('Abs_' + str(stack[i].cur_id))
        else:
            new_rep.append(stack[i])
    return new_rep

In [713]:
def get_emulated_results(stack_emulator, eq_dict, cur_iter_num, max_iter_num, cur_instruction_sequence):
    
    if cur_iter_num == max_iter_num:
        return
    pushX = lambda : stack_emulator.pushX(5 * cur_iter_num)
    pop = lambda : stack_emulator.pop()
    Xload = lambda : stack_emulator.Xload()
    # Xstore = lambda : stack_emulator.Xstore()
    dupX = lambda : stack_emulator.dupX(1)
    swapX = lambda : stack_emulator.swapX(1)
    
    stack_copy = list(stack_emulator.stack)
    
    # (Xstore, 'Xstore'),
    op_lst = [(pushX, 'pushX'), (pop, 'pop'), (Xload, 'Xload'), 
                                        (dupX, 'dupX'), (swapX, 'swapX')]
    
    for op in op_lst:
        try:
            op[0]()
        except:
            continue
        readable_stack_result = readable_stack(stack_emulator.stack)
        #print(readable_stack_result, cur_instruction_sequence + [op[1]])
        
        l = eq_dict.get(tuple(readable_stack_result), [])
        l.append(cur_instruction_sequence + [op[1]])
        eq_dict[tuple(readable_stack_result)] = l
        # assume try_all already restored the stack
        get_emulated_results(stack_emulator, eq_dict, cur_iter_num + 1, max_iter_num, cur_instruction_sequence + [op[1]])
        
        stack_emulator.stack = list(stack_copy)
    
    
    
def replace_with_predefined_patterns(eq_dict):
    def update_pattern(cur_pattern_seq, target_pattern):
        while True:
            old_pattern = target_pattern[0]
            new_pattern = target_pattern[1]
            start, end = find_sub_list(list(old_pattern), cur_pattern_seq)
            if start == -9999 and end == -9999:
                break
            cur_pattern_seq = cur_pattern_seq[:start] + list(new_pattern) + cur_pattern_seq[end + 1:]
        return cur_pattern_seq


    short_patterns_2 = [(('pushX', 'pop'), ()), (('dupX', 'pop'), ()), (('dupX', 'swapX'), ('dupX',)),
                       (('swapX', 'swapX'), ()), (('Xload', 'pop'), ('pop',))]
    short_patterns_3 = [(('pushX', 'Xload', 'pop'), ())]

    for k in eq_dict:
        l = list(eq_dict[k]['full_ops_lst'])
        for ind in range(len(l)):
            cur_pattern_seq = l[ind]
            cur_pattern_seq_old_val = list(cur_pattern_seq)
            while True:
                
                for target_pattern in short_patterns_2:
                    cur_pattern_seq = update_pattern(cur_pattern_seq, target_pattern)
                for target_pattern in short_patterns_3:
                    cur_pattern_seq = update_pattern(cur_pattern_seq, target_pattern)

                l[ind] = cur_pattern_seq
                
                if len(cur_pattern_seq_old_val) != len(cur_pattern_seq):
                    cur_pattern_seq_old_val = list(cur_pattern_seq)
                    continue
                if all([cur_pattern_seq_old_val[i] == cur_pattern_seq[i] for i in range(len(cur_pattern_seq_old_val))]):
                    break
                else:
                    cur_pattern_seq_old_val = list(cur_pattern_seq)
        eq_dict[k]['pattern_removed'] = l

            
def filter_out_monotonous_entries(eq_dict):
    for k in eq_dict:
        v = eq_dict[k]
        if len(v['pattern_removed']) > 1:
            v['is_monotonous'] = False
        else:
            v['is_monotonous'] = True
    t = {k:v for k,v in eq_dict.items() if len(v['pattern_removed']) > 1}
    assert all([not eq_dict[k]['is_monotonous'] for k in t])
    assert all([eq_dict[k]['is_monotonous'] for k in eq_dict if k not in t])
    print('number of stack status that have multiple sequences: ', len(t))

def remove_common_prefix(eq_dict):
    for k in eq_dict:
        if eq_dict[k]['is_monotonous']:
            continue
        v = eq_dict[k]['pattern_removed']
        print(v)
        print(eq_dict[k]['full_ops_lst'])
        cur_diff = find_diff(v)
        if cur_diff is None or all([len(i) == 0 for i in cur_diff]):
            eq_dict[k]['prefix_removed'] = [()]
            eq_dict[k]['is_monotonous'] = True
        else:
            eq_dict[k]['prefix_removed'] = cur_diff
        print(cur_diff)
        
def get_set_of_equivalent_ops(eq_dict):
    s = set()
    for k in eq_dict:
        if eq_dict[k]['is_monotonous']:
            continue
        v = eq_dict[k]['pattern_removed']
        new_pattern = []
        for i in v:
            if tuple(i) in new_pattern:
                continue
            else:
                new_pattern.append(tuple(i))
        s.add(tuple(new_pattern))
    return s



def is_homogeneous_tuple(tup):
    return all([i == tup[0] for i in tup])


def lift_dict(eq_dict):
    for k in eq_dict:
        eq_dict[k] = {'full_ops_lst': eq_dict[k]}
        
def find_diff(l):
    if all([len(i) == 0 for i in l]):
        return None
    if any([len(i) == 0 for i in l]):
        return [tuple(i) for i in l]
    
    ind = 0
    min_length = min([len(i) for i in l])
    while True:
        if ind == min_length:
            # reached the max length of one of the sequences
            break
        if all([i[ind] for i in l]):
            # current index is all same, move onto the next
            ind += 1
            continue
        else:
            # this means ind will be the index of difference
            break
    return [tuple(i[ind:]) for i in l]

In [801]:
stack_emulator = Emulator()
eq_dict = dict()
get_emulated_results(stack_emulator, eq_dict = eq_dict, cur_iter_num = 0, max_iter_num = 10, cur_instruction_sequence = [])

In [802]:
lift_dict(eq_dict)

In [803]:
replace_with_predefined_patterns(eq_dict)

In [804]:
filter_out_monotonous_entries(eq_dict)

number of stack status that have multiple sequences:  60170


In [805]:
# also updates is_monotonous
remove_common_prefix(eq_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'dupX', 'dupX'], ['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'dupX', 'dupX'], ['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'dupX', 'dupX']]
[['pushX', 'dupX', 'swapX', 'Xload', 'swapX', 'dupX', 'Xload', 'dupX', 'dupX'], ['pushX', 'dupX', 'swapX', 'Xload', 'swapX', 'dupX', 'Xload', 'dupX', 'dupX', 'swapX'], ['pushX', 'dupX', 'swapX', 'Xload', 'swapX', 'dupX', 'Xload', 'dupX', 'swapX', 'dupX']]
[(), (), ()]
[['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'swapX', 'pop'], ['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'swapX', 'pop']]
[['pushX', 'dupX', 'swapX', 'Xload', 'swapX', 'dupX', 'Xload', 'swapX', 'pop'], ['pushX', 'dupX', 'swapX', 'Xload', 'swapX', 'dupX', 'Xload', 'swapX', 'Xload', 'pop']]
[(), ()]
[['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'swapX', 'dupX'], ['pushX', 'dupX', 'Xload', 'swapX', 'dupX', 'Xload', 'swapX', 'dupX']]
[['pushX', 'dupX', 'swapX', 'Xload', 'swapX', 'dupX', 'Xloa

In [806]:
set_of_equivalent_ops = get_set_of_equivalent_ops(eq_dict)

In [815]:
len(eq_dict)

498671

In [807]:
set_of_equivalent_ops

{(('pushX', 'pushX', 'swapX', 'pop', 'pushX', 'swapX'),
  ('pushX', 'pushX', 'swapX'),
  ('pushX', 'Xload', 'Xload', 'pushX', 'swapX', 'pop', 'pushX', 'swapX')),
 (('pushX', 'pushX', 'pushX', 'Xload', 'swapX', 'pop'),
  ('pushX',
   'pushX',
   'pushX',
   'Xload',
   'swapX',
   'pushX',
   'Xload',
   'swapX',
   'pop',
   'pop'),
  ('pushX',
   'pushX',
   'pushX',
   'Xload',
   'swapX',
   'pushX',
   'swapX',
   'pop',
   'pop'),
  ('pushX',
   'pushX',
   'pushX',
   'Xload',
   'swapX',
   'Xload',
   'pushX',
   'swapX',
   'pop',
   'pop'),
  ('pushX',
   'pushX',
   'pushX',
   'Xload',
   'swapX',
   'dupX',
   'Xload',
   'swapX',
   'pop',
   'pop')),
 (('pushX', 'pushX', 'dupX', 'pushX', 'pushX', 'pushX'),
  ('pushX',
   'pushX',
   'dupX',
   'pushX',
   'pushX',
   'Xload',
   'pushX',
   'swapX',
   'pop',
   'pushX'),
  ('pushX',
   'pushX',
   'dupX',
   'pushX',
   'dupX',
   'Xload',
   'pushX',
   'swapX',
   'pop',
   'pushX'),
  ('pushX',
   'pushX',
   'dupX',

In [808]:
cost_dict = {'pushX': 3, 'dupX': 3, 'swapX': 3, 'pop': 2, 'Xstore': 2000, 'Xload': 2000}

for k in eq_dict:
    for type_lst in ['full_ops_lst', 'pattern_removed', 'prefix_removed']:
        if type_lst not in eq_dict[k]:
            continue
        for ind in range(len(eq_dict[k][type_lst])):
            cur_op_seq = eq_dict[k][type_lst][ind]
            print(cur_op_seq)
            cur_cost = 0
            for op in cur_op_seq:
                cur_cost += cost_dict[op]
            print(cur_op_seq, cur_cost)
            eq_dict[k][type_lst][ind] = (cur_op_seq, cur_cost)
    


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [809]:
def sort_by_min_gas(eq_dict):
    for k in eq_dict:
        for type_lst in ['full_ops_lst', 'pattern_removed', 'prefix_removed']:
            if type_lst not in eq_dict[k]:
                continue
            
            eq_dict[k][type_lst].sort(key = lambda x: x[1])

In [810]:
sort_by_min_gas(eq_dict)

In [811]:
for k in eq_dict:
    for type_lst in ['full_ops_lst']:
        if type_lst not in eq_dict[k]:
            continue
        if len(eq_dict[k][type_lst]) == 1:
            continue

        print(eq_dict[k][type_lst][0][1] - eq_dict[k][type_lst][-1][1])

-14005
-12005
-10005
-8005
-6005
-4005
-2005
-6
-1
-6
-6
-3
0
-1
-1
-1
-1
-3995
-4005
-4005
-2005
-1
-6
-2005
-1
-4000
-6
-6
-1997
-3
-3
-2000
0
0
0
-3
-2003
-1
-1
-1
-1
-4
-1
-1
-1
-1
-4000
-1
-3995
-3995
-3995
-3995
-7990
-8005
-8005
-6005
-3995
-4005
-6005
-3995
-8000
-4005
-6
-5992
-3
-1
-1
-1
-1
-2005
-1
-6
-1997
-1
-1
-1
-1
-6000
-3995
-4000
-4000
-2005
-1
-6
-2005
-6
-1
-6
-6
-3
-1997
-1997
-1997
-1997
-1997
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-3
0
-6
-3
0
0
0
0
0
-2000
0
0
-2000
0
-6003
0
-6
0
-3
-3
-4003
-7
-1
-1
-1
-1
-1998
-2003
-2003
-4
-1
-1
-1
-1
-1
-6
-3
-1
-1
-1
-1
-4
-1
-1
-1
-1
-1
-1
-1
-6000
-3995
-4000
-4000
-4
-1
-1
-5998
-3995
-3995
-3995
-3995
-3998
-3995
-3995
-3995
-3995
-7995
-3995
-7990
-7990
-7990
-7990
-11985
-16000
-12005
-10005
-7990
-8005
-10005
-7990
-12000
-8005
-6
-9987
-3
-3995
-3995
-3995
-3995
-6005
-3995
-4005
-5992
-3995
-3995
-3995
-3995
-10000
-7990
-8000
-8000
-6005
-3995
-4005
-2005
-6
-1
-6
-6
-3
-5992
-5992
-5992
-5992
-5992
-6
-3
-3
-2000
-3
-20

-4005
-4005
-2005
-1
-6
-2005
-1
-6
-6
-1997
-3
-3
-2000
0
0
0
-3
-2003
-1
-1
-1
-1
-4
-1
-1
-1
-1
-1
-6
-3
-1
-1
-1
-1
-2005
-1
-6
-1997
-1
-1
-1
-1
-6000
-3995
-4000
-4000
-2005
-1
-6
-2005
-6
-1
-6
-6
-3
-1997
-1997
-1997
-1997
-1997
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-3
0
-6
-3
-6
-3
-3
-6
-3
-5
-3
-2005
-6
-1
-6
-6
-3
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-4005
-2005
-6
-1
-6
-6
-3
0
-1
-1
-1
-1
-2005
-1
-6
-2005
-1
-4000
-6
-6
-1997
-3
-3
-3
-6
-3
-2005
-6
-1
-6
-6
-3
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-3
0
-6
-3
-2000
-2000
-6
-3
-3
-2005
-6
-1
-6
-6
-3
-6
-6
-3
-3
-3
-6
-3
-2005
-6
-1
-6
-6
-3
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-3
0
0
0
0
0
-3
-6
-3
-2005
-6
-1
-6
-6
-3
0
-6
-3
-3
-2000
-3
-2003
-3
-6
-3
-2000
-1997
-2000
-2000
-2005
-6
-1
-6
-6
-3
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-3
0
-6
-3
-4005
-2005
-6
-1
-6
-6
-3
0
-1
-1
-1
-1
-2005
-1
-6
-2005
-1
-6
-6
-1997
-3
-3
-2005
-1
-6
-6
-3
-6
-3
-2005
-6
-1
-6
-6
-3
-6
-3
-3
-2000
-3
-6
-3
-6
-3
-3
0
-6
-3
-6
-3
-3
-6
-3
-6
-3
-2005
-6
-1
-6
-6
-3
-6
-3
-

In [812]:
len(set_of_equivalent_ops)

2475

In [813]:
s = 0
for cur_Set in set_of_equivalent_ops:
    s += len(cur_Set)
print(s)

13222


In [761]:
13222/2475

5.342222222222222

In [ ]:
stack_emulator = Emulator()

In [72]:
Abstract_Value().cur_id

3

In [47]:
stack_emulator = Emulator()

In [48]:
for i in range(10):
    stack_emulator.pushX(i)

In [49]:
stack_emulator.dupX(5)

In [55]:
stack_emulator.swapX(1)

In [57]:
pop = stack_emulator.pop

In [59]:
pop()

In [60]:
stack_emulator.stack

[0, 1, 2, 3, 4, 5, 6, 7, 8, 5]

In [ ]:
Notes:
    
    1. out emulator only simulates the stack and unearths insights about opcode sequences, instead of 
    2. later on could put LLM as intermediate steps
    3. could reduce the optimization problem to a graph problem, and find e.g. the MST.
    4. could also expand the full tree and stop at a given level (otherwise too large instructions)

In [ ]:
limitations:
    
    1. existing bytecode issues
    
double optimization: first optimize source code and thne

In [ ]:
Whats new:

1. Use LLM to discover NEW source code level gas optimizations
2. Use LLM to discover NEW bytecode level gas optimizations (note: this probably wont quite work, 
       but could serve as a presentation of insights on the understanding of LLMs) (could be augmented
       with domain knowledge of gas costs)
3. Use implemented Stack to discover NEW bytecode level gas optimizations
       Could turn the problem to a searching problem in a tree/graph, but if complete search is possible..
            in other words, this helps with super long sequences, but not quite necessary yet.
       Could also add pre-tuning to prune known inefficient patterns
4. Use LLM to discover source code and bytecode optimizations, as a demonstration of the ability of 
       LLMs to understand and perform code analysis tasks (Could also use Python to help with this...)
       (could write this as a scary algorithm)
5. For the above, we also need to infuse various domain knowledge of gas, such as the cost of each 
       commonly seen operation.

What to do:
    1. look at how other gas optimization papers did evaluation (my idea is to 
          find some gas optimization cases from the wild, or compare to existing tools
          in terms of the amount of gas spent)
    2. 
(simplied a complex verification system to an empirical discovery methodology)
(could also use the previous code generation method as an evaluation or application)


In [ ]:
class OLDODLDOLDOLDOLDOKD_Emulator:
    
    def __init__(self):
        # right most is the top of the stack
        self.stack = []
        # [Abstract_Value() for i in range(5)]
        # self.max_length = 10000
    
    
        
    def pushX(self, value_id):
        self.stack.append(Abstract_Value(value_id))
        
    def pop(self):
        assert len(self.stack) >= 1
        self.stack.pop()
        
    def Xload(self):
        self.pop()
        self.pushX(Abstract_Value())
        
    def Xstore(self):
        self.pop()
        self.pop()
        
    def dupX(self, X):
        # 1 <= X <= 16
        assert len(self.stack) >= X
        self.stack.append(self.stack[-X])
        
    def swapX(self, X):
        assert len(self.stack) >= X + 1
        top_item = self.stack.pop()
        temp = self.stack[-X]
        self.stack[-X] = top_item
        self.stack.append(temp)  
       

    def abstract_remove_2_add_1(self, instruction_name):
        assert len(self.stack) >= 2

        top_item1 = self.stack.pop()
        top_item2 = self.stack.pop()

        new_expression = Abstract_Expression(instruction_name, [top_item1, top_item2])
        self.stack.append(new_expression)

    def abstract_remove_1_add_1(self, instruction_name):
        assert len(self.stack) >= 1

        top_item1 = self.stack.pop()

        new_expression = Abstract_Expression(instruction_name, [top_item1])
        self.stack.append(new_expression)

    def abstract_remove_None_add_1(self, instruction_name):
        
        top_item1 = self.stack.pop()

        new_expression = Abstract_Expression(instruction_name, [top_item1])
        self.stack.append(new_expression)


In [ ]:
        
        
# will need to check for tree equivalence
# not considering pushed values for generalizability and pattern caching
class Abstract_Expression:
    def __init__(self, cur_operation_name, children_lst):
        for child in children_lst:
            assert type(child) in [Abstract_Expression, Abstract_Value] 
        self.cur_operation_name = cur_operation_name
        self.children_lst = list(children_lst)